# Catboost Hyperparameter Sweep 20210920
Largely after https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu

# Setup

In [1]:
# two manual flags (ex-config)
COLAB = False
USE_GPU = True

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random
import multiprocessing
import pickle

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"sweep_catboost_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# handle Google Colab-specific library installation/updating
if COLAB:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
    # !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    # !pip install --upgrade xgboost

    # upgrade sklearn
    !pip install --upgrade scikit-learn

    # !pip install category_encoders
    !pip install catboost

    # lighgbm gpu compatible
    # !git clone --recursive https://github.com/Microsoft/LightGBM
    # ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
    
    # # this part is from https://github.com/rapidsai/gputreeshap/issues/24
    # !pip install cmake --upgrade
    # # !pip install sklearn --upgrade
    # !git clone --recursive https://github.com/dmlc/xgboost
    # %cd /content/xgboost
    # !mkdir build
    # %cd build
    # !cmake .. -DUSE_CUDA=ON
    # !make -j4
    # %cd /content/xgboost/python-package
    # !python setup.py install --use-cuda --use-nccl
    # !/opt/bin/nvidia-smi
    # !pip install shap
    

Now, non-stdlib imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from optuna.integration.wandb import WeightsAndBiasesCallback
# from wandb.xgboost import wandb_callback
# from wandb.lightgbm import wandb_callback
# from sklearn.impute import KNNImputer, StandardImputer
# import timm

import seaborn as sns

from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
# from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft


from optuna.samplers import TPESampler
import optuna
import catboost
from sklearn.utils import resample
import sklearn.metrics

Now, datapath setup

In [6]:
# # This is the code for reading the train.csv and converting it to a .feather file
# df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)
# df.index.name = None
# df.to_feather(path='./dataset_df.feather')

In [7]:
if COLAB:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/oct2021/')
    
else:
    # if on local machine
#     datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')  
    root = Path('/home/sf/code/kaggle/tabular_playgrounds/oct2021/')
    datapath = root/'datasets'
    edapath = root/'EDA'
    modelpath = root/'models'
    predpath = root/'preds'
    subpath = root/'submissions'
    
    for pth in [root, datapath, edapath, modelpath, predpath, subpath]:
        pth.mkdir(exist_ok=True)
    


In [8]:

# n_trials = int(1000)
SEED = 42

In [9]:
# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(SEED)

## Ex-Model Config

In [10]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
exmodel_config = {
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
#     'random_state': SEED,
#     'feature_generation': ['NaN_counts', 'SummaryStats', 'NaN_OneHots'],
#     'subsample': 1,
    'cross_val_strategy': KFold, # None for holdout, or the relevant sklearn class
    'kfolds': 5, # if 1, that means just doing holdout
    'test_size': 0.2,
#     'features_created': False,
#     'feature_creator': None,
}

# WandB Config

In [11]:
wandb_kwargs = {
    # wandb config
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'project': '202110_Kaggle_tabular_playground',
    'tags': ['sweep'],
    'notes': "Sweep for CatBoost using Optuna",
    'config': exmodel_config,
}

## Data Setup

In [12]:
train_source = datapath/'train.feather'
df = pd.read_feather(path=train_source)
df.index.name = 'id'
y_train = df.target
features = [x for x in df.columns if x != 'target']
X_train = df[features]
# X.index.name = 'id'
# y.index.name = 'id'
X = np.array(X_train)
y = np.array(y_train)

# del df, X_train, y_train


# exmodel_config['feature_count'] = len(X.columns)
exmodel_config['feature_count'] = X.shape[1]
exmodel_config['instance_count'] = X.shape[0]

# exmodel_config['feature_generator'] = None
# exmodel_config['feature_generator'] = "Summary statistics"

exmodel_config['train_source'] = str(train_source)

In [13]:
test_source = datapath/'test.feather'
exmodel_config['test_source'] = str(test_source)
X_test = pd.read_feather(path=test_source)
# X_test = X_test.iloc[:, 1:]

In [14]:
X_test = np.array(X_test)

# Experiment setup

In [15]:
# wandb_kwargs = {
#     # wandb config:
#     'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
#     'project': '202109_Kaggle_tabular_playground',
#     'tags': ['sweep'],
#     'notes': "Sweep for CatBoost using Optuna",
#     'config': exmodel_config,
# }

A basis for the experimental ranges, taken from [here](https://www.kaggle.com/vishwas21/tps-sep-21-3-level-custom-stacking):

```python
catb_params = {
    'iterations': 15585, 
    'objective': 'CrossEntropy', 
    'bootstrap_type': 'Bernoulli', 
    'od_wait': 1144, 
    'learning_rate': 0.023575206684596582, 
    'reg_lambda': 36.30433203563295, 
    'random_strength': 43.75597655616195, 
    'depth': 7, 
    'min_data_in_leaf': 11, 
    'leaf_estimation_iterations': 1, 
    'subsample': 0.8227911142845009,
    'task_type' : 'GPU',
    'devices' : '0',
    'verbose' : 0
}
```

In [16]:
# originally from https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu
def objective(trial):
    # split the (original Kaggle training) data into partitions
    # if study.best_trial:
    #     print("Dumping best params, which are:")
    #     print(str(study.best_trial.params))
    #     dump(study.best_trial.params, filename=datapath/'optuna_catboost_best_20210920.joblib')
       
    # else:
    #     print("No best study yet")
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=int(SEED), shuffle=True)
    # create wrappers for the training and validation partitions
    train_pool = catboost.Pool(X_train, y_train)
    valid_pool = catboost.Pool(X_valid, y_valid)
    
    # experimental parameters
    params = {
        'iterations' : trial.suggest_int('iterations', 2000, 30000),                         
        'depth' : trial.suggest_int('depth', 3, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.5),               
        'random_strength': trial.suggest_int('random_strength', 0, 100), 
#         'objective': trial.suggest_categorical('objective', ['Logloss', 'CrossEntropy']),
#         'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['MVC', 'Bernoulli']),#, 'Poisson']),
        'od_wait': trial.suggest_int('od_wait', 20, 2000),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 2, 70), # aka l2_leaf_reg
        'border_count': trial.suggest_int('border_count', 50, 275),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20), # aka min_data_in_leaf
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 5),
        # 'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
#         'subsample': trial.suggest_uniform('subsample', 0.5, 1),
#         'learning_rate' :trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        # 'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#         'max_leaves': trial.suggest_int('max_leaves', 32, 128)
    }
    device = "GPU" if USE_GPU else "CPU"
    # instantiate the model, with some parameters locked in, and experimnental ones passed via splat 
    model = CatBoostClassifier(
        # loss_function="Logloss",
        eval_metric="AUC",
        task_type=device,
        objective='Logloss',
#         l2_leaf_reg=50,
        random_seed=SEED,
        verbose=False,
#         border_count=64,
        **params
    )        
#     wandb.init(wandb_kwargs)
    # fit the model on the training set -- note the use of the catboost.Pool instance
    model.fit(train_pool)
    # generate predictions
    preds = model.predict_proba(valid_pool)[:,1]
    # rounds to the nearest integer, and the nearest even in case of _.5s
    # pred_labels = np.rint(preds)
    # calls sklearn.utils.resample... not entirely sure why
    # y_pred_boot = resample(pred_labels, n_samples = len(train_y))
    # Evaluation
    # ROC_AUC_Score = roc_auc_score(train_y, y_pred_boot)
    valid_auc = roc_auc_score(y_valid, preds)
    
    print('ROC AUC Score of CatBoost =', valid_auc)
    wandb.log({'valid_AUC': valid_auc,
            #    'catboost_params': str(model.get_all_params())
              })
#     wandb.finish()
    return valid_auc

In [17]:
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

<ipython-input-17-69ea9289a2cf>:1: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [18]:
study = optuna.create_study(direction = "maximize", sampler = TPESampler(seed=int(SEED)), study_name='CatBoost_20211001')
# setting n_jobs=1 bc wandb apparently requires this


[I 2021-10-01 13:45:41,774] A new study created in memory with name: CatBoost_20211001


In [19]:
for x in range(100):
    study.optimize(objective, n_trials = 1, callbacks = [wandbc]) #n_jobs = multiprocessing.cpu_count())
    dump(study, filename=datapath/f'optuna_catboost_study_thru_trial{x}_20211001.joblib')

[I 2021-10-01 13:54:35,114] Trial 0 finished with value: 0.8503879890111978 and parameters: {'iterations': 12487, 'depth': 10, 'learning_rate': 0.09454306819536169, 'random_strength': 60, 'od_wait': 329, 'reg_lambda': 12.60762738286178, 'border_count': 63, 'min_child_samples': 18, 'leaf_estimation_iterations': 4}. Best is trial 0 with value: 0.8503879890111978.
wandb: WARNING Step must only increase in log calls.  Step 0 < 1; dropping {'iterations': 12487, 'depth': 10, 'learning_rate': 0.09454306819536169, 'random_strength': 60, 'od_wait': 329, 'reg_lambda': 12.60762738286178, 'border_count': 63, 'min_child_samples': 18, 'leaf_estimation_iterations': 4, 'value': 0.8503879890111978}.


ROC AUC Score of CatBoost = 0.8503879890111978


[I 2021-10-01 13:59:02,805] Trial 1 finished with value: 0.8350362294833469 and parameters: {'iterations': 21826, 'depth': 3, 'learning_rate': 0.41472250004816347, 'random_strength': 84, 'od_wait': 440, 'reg_lambda': 14.364097770082843, 'border_count': 91, 'min_child_samples': 7, 'leaf_estimation_iterations': 3}. Best is trial 0 with value: 0.8503879890111978.
wandb: WARNING Step must only increase in log calls.  Step 1 < 2; dropping {'iterations': 21826, 'depth': 3, 'learning_rate': 0.41472250004816347, 'random_strength': 84, 'od_wait': 440, 'reg_lambda': 14.364097770082843, 'border_count': 91, 'min_child_samples': 7, 'leaf_estimation_iterations': 3, 'value': 0.8350362294833469}.


ROC AUC Score of CatBoost = 0.8350362294833469


[I 2021-10-01 14:03:16,570] Trial 2 finished with value: 0.8558281969273086 and parameters: {'iterations': 14094, 'depth': 5, 'learning_rate': 0.044809759182149515, 'random_strength': 14, 'od_wait': 598, 'reg_lambda': 26.912605343971038, 'border_count': 153, 'min_child_samples': 16, 'leaf_estimation_iterations': 1}. Best is trial 2 with value: 0.8558281969273086.
wandb: WARNING Step must only increase in log calls.  Step 2 < 3; dropping {'iterations': 14094, 'depth': 5, 'learning_rate': 0.044809759182149515, 'random_strength': 14, 'od_wait': 598, 'reg_lambda': 26.912605343971038, 'border_count': 153, 'min_child_samples': 16, 'leaf_estimation_iterations': 1, 'value': 0.8558281969273086}.


...

ROC AUC Score of CatBoost = 0.856718294380942


[I 2021-10-02 10:02:07,842] Trial 98 finished with value: 0.8567280151709886 and parameters: {'iterations': 29555, 'depth': 9, 'learning_rate': 0.003505819774785112, 'random_strength': 8, 'od_wait': 1973, 'reg_lambda': 24.717646416657928, 'border_count': 161, 'min_child_samples': 14, 'leaf_estimation_iterations': 3}. Best is trial 95 with value: 0.8568423435731147.
wandb: WARNING Step must only increase in log calls.  Step 98 < 99; dropping {'iterations': 29555, 'depth': 9, 'learning_rate': 0.003505819774785112, 'random_strength': 8, 'od_wait': 1973, 'reg_lambda': 24.717646416657928, 'border_count': 161, 'min_child_samples': 14, 'leaf_estimation_iterations': 3, 'value': 0.8567280151709886}.


ROC AUC Score of CatBoost = 0.8567280151709886


[I 2021-10-02 10:19:58,331] Trial 99 finished with value: 0.8567797303969964 and parameters: {'iterations': 28771, 'depth': 9, 'learning_rate': 0.007947948720605974, 'random_strength': 5, 'od_wait': 1845, 'reg_lambda': 46.52812648470962, 'border_count': 176, 'min_child_samples': 14, 'leaf_estimation_iterations': 1}. Best is trial 95 with value: 0.8568423435731147.
wandb: WARNING Step must only increase in log calls.  Step 99 < 100; dropping {'iterations': 28771, 'depth': 9, 'learning_rate': 0.007947948720605974, 'random_strength': 5, 'od_wait': 1845, 'reg_lambda': 46.52812648470962, 'border_count': 176, 'min_child_samples': 14, 'leaf_estimation_iterations': 1, 'value': 0.8567797303969964}.


ROC AUC Score of CatBoost = 0.8567797303969964


In [20]:
dump(study.best_trial.params, filename=datapath/'optuna_catboost_best_20211001.joblib')
# pickle.dump(study.best_trial.params, open('CatBoost_Hyperparameter.pickle', 'wb'))
# print('CatBoost Hyperparameter:', study.best_trial.params)

['/home/sf/code/kaggle/tabular_playgrounds/oct2021/datasets/optuna_catboost_best_20211001.joblib']

In [21]:
wandb.log({'catboost_params': study.best_trial.params})
wandb.finish()

In [22]:
study.best_trial.params

{'iterations': 29338,
 'depth': 9,
 'learning_rate': 0.004769831650275205,
 'random_strength': 7,
 'od_wait': 1968,
 'reg_lambda': 28.435563240493586,
 'border_count': 162,
 'min_child_samples': 14,
 'leaf_estimation_iterations': 1}